In [1]:
import requests
import json
import time
import praw
from bs4 import BeautifulSoup

# Twitter

In [8]:
# Get BEARER_TOKEN from key.json
with open('key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [15]:
def get_tweets(search, num_tweets = 5):
	# Create the url
	url = 'https://api.twitter.com/1.1/search/tweets.json?result_type=popular&q='+ search + '&max_results=' + str(num_tweets)
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [18]:
wheels_json = get_tweets('wheel', 100)
doors_json = get_tweets('door', 100)

In [19]:
# Write to json file
with open('data/twitter/wheels.json', 'w') as f:
	json.dump(wheels_json, f)
with open('data/twitter/doors.json', 'w') as f:
	json.dump(doors_json, f)

# Reddit

In [8]:
# Build reddit object

reddit = praw.Reddit(
    client_id="r3dlLRYKcBN9-JRoyvi1Xw",
    client_secret="YfKwRqhLjS1darC2TL5QnrzDe0S_sQ",
    user_agent="hiiiiiiiiii",
)

print(reddit.read_only)

True


In [31]:
wheels_reddit = reddit.subreddit("all").search("wheel", limit=1000)
doors_reddit = reddit.subreddit("all").search("door" , limit=1000)

In [32]:
# Write to csv file	
with open('data/reddit/wheels_reddit.csv', 'w') as f:
	f.write('"title"\n')
	for submission in wheels_reddit:
		f.write('"' + submission.title.replace('"', '""') + '"\n')
  
with open('data/reddit/doors_reddit.csv', 'w') as f:
	f.write('"title"\n')
	for submission in doors_reddit:
		f.write('"' + submission.title.replace('"', '""') + '"\n')

# Wikipedia

In [2]:
def get_wikipedia_data(search, num_posts = 5):
	# Create the url for the search also get the posts text
	url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=' + search + '&format=json&srlimit=' + str(num_posts)
	# Make the request
	response = requests.get(url)
	return response.json()

In [3]:
wheels_wiki = get_wikipedia_data('wheels', 100)
doors_wiki = get_wikipedia_data('doors', 100)

In [5]:
with open('data/wiki/wheels_wiki.json', 'w') as f:
	json.dump(wheels_wiki, f)
with open('data/wiki/doors_wiki.json', 'w') as f:
	json.dump(doors_wiki, f)

In [6]:
# Get the pages from wikipedia
for page in wheels_wiki['query']['search']:
	url = 'https://en.wikipedia.org/wiki/' + page['title']
	response = requests.get(url)
	soup = BeautifulSoup(response.text, 'html.parser')
	file_name = 'data/wiki/wheels_wiki_pages/' + page['title'] + '.txt'
	# Make file name compatible with windows
	file_name = file_name.replace(' ', '_').replace(':', '_').replace('\\', '_').replace('"', '_').replace('?', '_').replace('*', '_').replace('|', '_').replace('<', '_').replace('>', '_')
	with open(file_name, 'w') as f:
		f.write(soup.get_text())
	time.sleep(0.1)

In [7]:
for page in doors_wiki['query']['search']:
	url = 'https://en.wikipedia.org/wiki/' + page['title']
	response = requests.get(url)
	soup = BeautifulSoup(response.text, 'html.parser')
 
	file_name = 'data/wiki/doors_wiki_pages/' + page['title'] + '.txt'
	file_name = file_name.replace(' ', '_').replace(':', '_').replace('\\', '_').replace('"', '_').replace('?', '_').replace('*', '_').replace('|', '_').replace('<', '_').replace('>', '_')
	with open(file_name, 'w') as f:
		f.write(soup.get_text())
	time.sleep(0.1)

# Download models

In [26]:
import gensim.downloader

In [27]:
model_twitter = gensim.downloader.load("glove-twitter-200")
model_twitter.save("glove_twitter_200.model")


In [28]:
model_wiki = gensim.downloader.load("glove-wiki-gigaword-300")
model_wiki.save("glove_wiki_gigaword_300.model")

In [29]:
model_google = gensim.downloader.load("word2vec-google-news-300")
model_google.save("word2vec_google_news_300.model")

Error: Canceled future for execute_request message before replies were done